# Multinomial Naive bayes

In [1]:
#import library packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
import warnings
warnings.filterwarnings("ignore") 

In [3]:
psy = pd.read_csv('Youtube01-Psy.csv')
katty = pd.read_csv('Youtube02-KatyPerry.csv')
Lmfao = pd.read_csv('Youtube03-LMFAO.csv')
Eminem = pd.read_csv('Youtube04-Eminem.csv')
shakira = pd.read_csv('Youtube04-Eminem.csv')

In [4]:
data=pd.concat([psy,katty,Lmfao,Eminem,shakira])

In [5]:
del data['DATE']
del data['COMMENT_ID']
del data['AUTHOR']

In [6]:
data

,CONTENT,CLASS
0,"Huh, anyway check out this you[tube] channel: ...",1
1,Hey guys check out my new channel and our firs...,1
2,just for test I have to say murdev.com,1
3,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,watch?v=vtaRGgvGtWQ Check this out .﻿,1
...,...,...
443,SUBSCRIBE TO MY CHANNEL X PLEASE!. SPARE,1
444,Check out my videos guy! :) Hope you guys had ...,1
445,3 yrs ago I had a health scare but thankfully ...,1
446,Rihanna looks so beautiful with red hair ;)﻿,0


In [7]:
data.duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
443     True
444     True
445     True
446     True
447     True
Length: 2034, dtype: bool

In [8]:
data.duplicated().sum()

594

In [9]:
data=data.drop_duplicates()

In [10]:
data.shape

(1440, 2)

In [11]:
data.isnull().sum()

CONTENT    0
CLASS      0
dtype: int64

In [12]:
data.reset_index(inplace=True)

In [13]:
# Data cleaning and preprocessing

import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

corpus=[]
for i in range(0, len(data)):
    review=re.sub('^[a-zA-Z][0-9]',' ', str(data['CONTENT'][i]))
    review=review.lower()
    review=review.split()

    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

corpus

['huh, anyway check you[tube] channel: kobyoshi02',
 "hey guy check new channel first vid us monkeys!!! i'm monkey white shirt,pleas leav like comment pleas subscribe!!!!",
 'test say murdev.com',
 'shake sexi ass channel enjoy ^_^ \ufeff',
 'watch?v=vtarggvgtwq check .\ufeff',
 'hey, check new website!! site kid stuff. kidsmediausa . com',
 'subscrib channel \ufeff',
 'turn mute soon came want check views...\ufeff',
 'check channel funni videos!!\ufeff',
 'u should.d check channel tell next!\ufeff',
 'hey subscrib me\ufeff',
 'start read stop. subscrib within one day entir famili die want stay aliv subscrib right now.\ufeff',
 'https://twitter.com/gbphotographygb\ufeff',
 'subscrib like comment\ufeff',
 'pleas like :d https://premium.easypromosapp.com/voteme/19924/616375350\ufeff',
 "hello! like gaming, art videos, scientif experiments, tutorials, lyric videos, much, much that? pleas check channel subscrib it, we'v started, soon hope abl cover expectations... also check we'v got far!\

In [15]:
df=data

In [16]:
type(df['CONTENT'].loc[100])

str

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1440 entries, 0 to 1439
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   index    1440 non-null   int64 
 1   CONTENT  1440 non-null   object
 2   CLASS    1440 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 33.9+ KB


In [18]:
# Creating the TFIDF model
from sklearn.feature_extraction.text import TfidfVectorizer
tv=TfidfVectorizer(max_features=3000,ngram_range=(1,2))
X=tv.fit_transform(corpus).toarray()

In [19]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [20]:
X.shape

(1440, 3000)

In [21]:
y=pd.get_dummies(df['CONTENT'])
y=y.iloc[:,1].values

In [22]:

y

array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)

In [23]:
# Since data is imbalanced
# Trying over sampling

from imblearn.over_sampling import RandomOverSampler

rs=RandomOverSampler()
X,y=rs.fit_resample(X,y)

X.shape,y.shape

((2878, 3000), (2878,))

In [24]:

# Train Test Split

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [25]:
from sklearn.naive_bayes import MultinomialNB
MNB = MultinomialNB()
MNB.fit(X_train,y_train)

MultinomialNB()

In [26]:
predict = MNB.predict(X_test)

In [27]:
from sklearn.metrics import accuracy_score
print('Accuracy of MultinomialNB',accuracy_score(y_test,predict)*100)

Accuracy of MultinomialNB 97.74305555555556


In [28]:
from sklearn.metrics import confusion_matrix
print('Confusuion matrix of MultinomialNB\n',confusion_matrix(y_test,predict))

Confusuion matrix of MultinomialNB
 [[292  13]
 [  0 271]]


In [29]:
from sklearn.metrics import classification_report
print('Classification report of MultinomialNB\n\n',classification_report(y_test,predict))

Classification report of MultinomialNB

               precision    recall  f1-score   support

           0       1.00      0.96      0.98       305
           1       0.95      1.00      0.98       271

    accuracy                           0.98       576
   macro avg       0.98      0.98      0.98       576
weighted avg       0.98      0.98      0.98       576

